In [1]:
import time
import warnings
import numpy as np
from snnpy import *
from sklearn.neighbors import BallTree
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree
from bf_search import *
from tqdm import tqdm

leaf_size = 40
warnings.filterwarnings("ignore")

### size (ndim=2)

In [2]:
n_samples = 3000
unit_inc = 3000
n_dim =  2
Rlist = [0.1, 0.3, 0.5, 0.7]


index_time_kd = list()
index_time_bl = list()
index_time_sn = list()

run_time_bf1 = list()
run_time_bf2 = list()
run_time_kd = list()
run_time_bl = list()
run_time_sn = list()

rng = np.random.RandomState(0)
xrange = np.arange(n_samples, n_samples + 10*unit_inc, unit_inc)
for size in tqdm(range(len(xrange))):
    i = xrange[size]
    data = rng.random_sample((2*i, n_dim)) 
    data = (data - data.mean(axis=0))/data.std(axis=0)
    X = data[:i]
    Query = data[i:]
    
    bf1et = 0
    bf2et = 0
    kd_id_et = 0
    kd_qy_et = 0 
    bl_id_et = 0
    bl_qy_et = 0
    sn_id_et = 0
    sn_qy_et = 0
    
    for R in Rlist:
        st = time.time()
        neigh = NearestNeighbors(radius=R, algorithm='brute')
        neigh.fit(X)
        for j in range(Query.shape[0]): 
            ind = neigh.radius_neighbors(
               Query[j:j+1], radius=R, return_distance=False
            )
        bf1et += time.time() - st

        st = time.time()
        for j in range(Query.shape[0]): 
            ind = bf_radius_fairness(Query[j], X, R, return_distance=False)
        bf2et += time.time() - st

        # KDtree
        st = time.time()
        kdtree = KDTree(X, leaf_size=leaf_size)       
        kd_id_et += time.time() - st
        
        
        st = time.time()
        for j in range(Query.shape[0]): 
            kdind = kdtree.query_radius(Query[j:j+1], r=R, return_distance=False) 
        kd_qy_et += time.time() - st
        
        # Balltree
        st = time.time()
        bltree = BallTree(X, leaf_size=leaf_size)
        bl_id_et += time.time() - st
        
        st = time.time()
        for j in range(Query.shape[0]): 
            blind = bltree.query_radius(Query[j:j+1], r=R, return_distance=False)  
        bl_qy_et += time.time() - st
        
        # snn
        st = time.time()
        snn = build_snn_model(X)
        sn_id_et += time.time() - st
        
        st = time.time()
        for j in range(Query.shape[0]): 
            sind = snn.radius_single_query(Query[j], R)
        sn_qy_et += time.time() - st
        
        
    print("brute force 1 query time:", bf1et/ len(Rlist))
    print("brute force 2 query time:", bf2et/ len(Rlist))
    print("KDtree index:", kd_id_et/ len(Rlist))
    print("KDtree query time:", kd_qy_et/ len(Rlist))
    print("Balltree index:", bl_id_et/ len(Rlist))
    print("Balltree query time:", bl_qy_et/ len(Rlist))
    print("SNN index:", sn_id_et/ len(Rlist))
    print("SNN query time:", sn_qy_et/ len(Rlist))
    
    index_time_kd.append( kd_id_et / len(Rlist))
    run_time_bf1.append(bf1et / len(Rlist))
    run_time_bf2.append(bf2et / len(Rlist))
    run_time_kd.append( kd_qy_et / len(Rlist))
    index_time_bl.append( bl_id_et / len(Rlist))
    run_time_bl.append( bl_qy_et / len(Rlist))
    index_time_sn.append(sn_id_et / len(Rlist))
    run_time_sn.append(sn_qy_et/ len(Rlist))
    print()

 10%|█         | 1/10 [00:05<00:47,  5.30s/it]

brute force 1 query time: 0.6879978775978088
brute force 2 query time: 0.20335817337036133
KDtree index: 0.0011867880821228027
KDtree query time: 0.1614854335784912
Balltree index: 0.0008478164672851562
Balltree query time: 0.14854729175567627
SNN index: 0.0011959075927734375
SNN query time: 0.11953014135360718



 20%|██        | 2/10 [00:18<01:21, 10.20s/it]

brute force 1 query time: 1.7630468606948853
brute force 2 query time: 0.6749171018600464
KDtree index: 0.0020766258239746094
KDtree query time: 0.3566626310348511
Balltree index: 0.0016753077507019043
Balltree query time: 0.32474613189697266
SNN index: 0.0015149712562561035
SNN query time: 0.281421422958374



 30%|███       | 3/10 [00:43<01:57, 16.82s/it]

...

brute force 1 query time: 15.338769853115082
brute force 2 query time: 9.253592789173126
KDtree index: 0.008931279182434082
KDtree query time: 2.0739572048187256
Balltree index: 0.008131742477416992
Balltree query time: 1.7861424088478088
SNN index: 0.0043312907218933105
SNN query time: 2.2049666047096252



 90%|█████████ | 9/10 [09:47<01:45, 105.43s/it]

brute force 1 query time: 19.250915050506592
brute force 2 query time: 11.800126552581787
KDtree index: 0.010350525379180908
KDtree query time: 2.4286020398139954
Balltree index: 0.009354054927825928
Balltree query time: 2.114581048488617
SNN index: 0.004893898963928223
SNN query time: 2.6850820183753967



100%|██████████| 10/10 [12:50<00:00, 77.07s/it] 

brute force 1 query time: 22.647675335407257
brute force 2 query time: 14.51008516550064
KDtree index: 0.011499345302581787
KDtree query time: 2.808230400085449
Balltree index: 0.01030033826828003
Balltree query time: 2.453445017337799
SNN index: 0.005533933639526367
SNN query time: 3.259497106075287



In [3]:
### save the data

index_time_kd = np.array(index_time_kd)
index_time_bl = np.array(index_time_bl) 
index_time_sn = np.array(index_time_sn) 

run_time_bf1 = np.array(run_time_bf1)
run_time_bf2 = np.array(run_time_bf2)
run_time_kd = np.array(run_time_kd)
run_time_bl = np.array(run_time_bl)
run_time_sn = np.array(run_time_sn)


with open('result/query_r/size/index_time_kd_d2.npy', 'wb') as f:
    np.save(f, index_time_kd)

with open('result/query_r/size/index_time_bl_d2.npy', 'wb') as f:
    np.save(f, index_time_bl)

with open('result/query_r/size/index_time_sn_d2.npy', 'wb') as f:
    np.save(f, index_time_sn)
    
with open('result/query_r/size/run_time_bf1_d2.npy', 'wb') as f:
    np.save(f, run_time_bf1)

with open('result/query_r/size/run_time_bf2_d2.npy', 'wb') as f:
    np.save(f, run_time_bf2)

with open('result/query_r/size/run_time_kd_d2.npy', 'wb') as f:
    np.save(f, run_time_kd)

with open('result/query_r/size/run_time_bl_d2.npy', 'wb') as f:
    np.save(f, run_time_bl)
    
with open('result/query_r/size/run_time_sn_d2.npy', 'wb') as f:
    np.save(f, run_time_sn)
    

### size (ndim=50)

In [4]:
n_samples = 3000
unit_inc = 3000
n_dim =  50
Rlist = [8, 10, 12, 14]


index_time_kd = list()
index_time_bl = list()
index_time_sn = list()

run_time_bf1 = list()
run_time_bf2 = list()
run_time_kd = list()
run_time_bl = list()
run_time_sn = list()

rng = np.random.RandomState(0)
xrange = np.arange(n_samples, n_samples + 10*unit_inc, unit_inc)
for size in tqdm(range(len(xrange))):
    i = xrange[size]
    data = rng.random_sample((2*i, n_dim)) 
    data = (data - data.mean(axis=0))/data.std(axis=0)
    X = data[:i]
    Query = data[i:]
     
    bf1et = 0
    bf2et = 0
    kd_id_et = 0
    kd_qy_et = 0 
    bl_id_et = 0
    bl_qy_et = 0
    sn_id_et = 0
    sn_qy_et = 0
    
    for R in Rlist:
        st = time.time()
        neigh = NearestNeighbors(radius=R, algorithm='brute')
        neigh.fit(X)
        for j in range(Query.shape[0]): 
            ind = neigh.radius_neighbors(
               Query[j:j+1], radius=R, return_distance=False
            )
        bf1et += time.time() - st

        st = time.time()
        for j in range(Query.shape[0]): 
            ind = bf_radius_fairness(Query[j], X, R, return_distance=False)
        bf2et += time.time() - st

        # KDtree
        st = time.time()
        kdtree = KDTree(X, leaf_size=leaf_size)       
        kd_id_et += time.time() - st
        
        
        st = time.time()
        for j in range(Query.shape[0]): 
            kdind = kdtree.query_radius(Query[j:j+1], r=R, return_distance=False) 
        kd_qy_et += time.time() - st
        
        # Balltree
        st = time.time()
        bltree = BallTree(X, leaf_size=leaf_size)
        bl_id_et += time.time() - st
        
        st = time.time()
        for j in range(Query.shape[0]): 
            blind = bltree.query_radius(Query[j:j+1], r=R, return_distance=False)  
        bl_qy_et += time.time() - st
        
        # snn
        st = time.time()
        snn = build_snn_model(X)
        sn_id_et += time.time() - st
        
        st = time.time()
        for j in range(Query.shape[0]): 
            sind = snn.radius_single_query(Query[j], R)
        sn_qy_et += time.time() - st
        
        
    print("brute force 1 query time:", bf1et/ len(Rlist))
    print("brute force 2 query time:", bf2et/ len(Rlist))
    print("KDtree index:", kd_id_et/ len(Rlist))
    print("KDtree query time:", kd_qy_et/ len(Rlist))
    print("Balltree index:", bl_id_et/ len(Rlist))
    print("Balltree query time:", bl_qy_et/ len(Rlist))
    print("SNN index:", sn_id_et/ len(Rlist))
    print("SNN query time:", sn_qy_et/ len(Rlist))
    
    index_time_kd.append(kd_id_et / len(Rlist))
    run_time_bf1.append(bf1et / len(Rlist))
    run_time_bf2.append(bf2et / len(Rlist))
    run_time_kd.append( kd_qy_et / len(Rlist))
    index_time_bl.append( bl_id_et / len(Rlist))
    run_time_bl.append( bl_qy_et / len(Rlist))
    index_time_sn.append(sn_id_et / len(Rlist))
    run_time_sn.append(sn_qy_et / len(Rlist))
    print()

 10%|█         | 1/10 [00:19<02:53, 19.24s/it]

brute force 1 query time: 1.3037661910057068
brute force 2 query time: 0.7395937442779541
KDtree index: 0.011156439781188965
KDtree query time: 1.5375614762306213
Balltree index: 0.007027685642242432
Balltree query time: 0.8262796998023987
SNN index: 0.0031054019927978516
SNN query time: 0.37819039821624756



 20%|██        | 2/10 [01:27<06:26, 48.30s/it]

brute force 1 query time: 3.835493266582489
brute force 2 query time: 2.8898231983184814
KDtree index: 0.025729894638061523
KDtree query time: 5.924376487731934
Balltree index: 0.01630955934524536
Balltree query time: 3.019830346107483
SNN index: 0.0058748722076416016
SNN query time: 1.436510443687439



 30%|███       | 3/10 [03:46<10:26, 89.57s/it]

...

brute force 1 query time: 47.26668077707291
brute force 2 query time: 38.344806134700775
KDtree index: 0.13287150859832764
KDtree query time: 91.41273123025894
Balltree index: 0.08741980791091919
Balltree query time: 46.024673104286194
SNN index: 0.02023792266845703
SNN query time: 21.64852273464203



 90%|█████████ | 9/10 [1:12:25<13:41, 821.56s/it]

brute force 1 query time: 61.382205069065094
brute force 2 query time: 49.941778898239136
KDtree index: 0.14984172582626343
KDtree query time: 111.3689147233963
Balltree index: 0.09907758235931396
Balltree query time: 58.862460911273956
SNN index: 0.021391689777374268
SNN query time: 27.75183594226837



100%|██████████| 10/10 [1:39:37<00:00, 597.73s/it] 

brute force 1 query time: 83.29248046875
brute force 2 query time: 65.53899598121643
KDtree index: 0.16860413551330566
KDtree query time: 141.1906782388687
Balltree index: 0.11163163185119629
Balltree query time: 81.40453153848648
SNN index: 0.023865103721618652
SNN query time: 36.23881542682648



In [5]:
### save the data

index_time_kd = np.array(index_time_kd)
index_time_bl = np.array(index_time_bl) 
index_time_sn = np.array(index_time_sn) 

run_time_bf1 = np.array(run_time_bf1)
run_time_bf2 = np.array(run_time_bf2)
run_time_kd = np.array(run_time_kd)
run_time_bl = np.array(run_time_bl)
run_time_sn = np.array(run_time_sn)


with open('result/query_r/size/index_time_kd.npy', 'wb') as f:
    np.save(f, index_time_kd)

with open('result/query_r/size/index_time_bl.npy', 'wb') as f:
    np.save(f, index_time_bl)

with open('result/query_r/size/index_time_sn.npy', 'wb') as f:
    np.save(f, index_time_sn)
    
with open('result/query_r/size/run_time_bf1.npy', 'wb') as f:
    np.save(f, run_time_bf1)

with open('result/query_r/size/run_time_bf2.npy', 'wb') as f:
    np.save(f, run_time_bf2)

with open('result/query_r/size/run_time_kd.npy', 'wb') as f:
    np.save(f, run_time_kd)

with open('result/query_r/size/run_time_bl.npy', 'wb') as f:
    np.save(f, run_time_bl)
    
with open('result/query_r/size/run_time_sn.npy', 'wb') as f:
    np.save(f, run_time_sn)
    

### dimensions

In [6]:
n_samples = 10000
unit_inc = 30
Rlist = [2.5, 9, 15.5, 22]
rng = np.random.RandomState(0)


index_time_kd = list()
index_time_bl = list()
index_time_sn = list()

run_time_bf1 = list()
run_time_bf2 = list()
run_time_kd = list()
run_time_bl = list()
run_time_sn = list()

xrange = np.arange(2, 2 + 10*unit_inc, unit_inc)
for dim in tqdm(range(len(xrange))):
    qm = 0
    i = xrange[dim]
    data = rng.random_sample((2*n_samples, i)) 
    data = (data - data.mean(axis=0))/data.std(axis=0)
    X = data[:n_samples]
    Query = data[n_samples:]
    
    bf1et = 0
    bf2et = 0
    kd_id_et = 0
    kd_qy_et = 0 
    bl_id_et = 0
    bl_qy_et = 0
    sn_id_et = 0
    sn_qy_et = 0
    
    for R in Rlist:
        st = time.time()
        neigh = NearestNeighbors(radius=R, algorithm='brute')
        neigh.fit(X)
        for j in range(Query.shape[0]): 
            ind = neigh.radius_neighbors(
               Query[j:j+1], radius=R, return_distance=False
            )
        bf1et += time.time() - st

        st = time.time()
        for j in range(Query.shape[0]): 
            ind = bf_radius_fairness(Query[j], X, R, return_distance=False)
        bf2et += time.time() - st

        # KDtree
        st = time.time()
        kdtree = KDTree(X, leaf_size=leaf_size)       
        kd_id_et += time.time() - st
        
        
        st = time.time()
        for j in range(Query.shape[0]): 
            kdind = kdtree.query_radius(Query[j:j+1], r=R, return_distance=False) 
        kd_qy_et += time.time() - st
        
        # Balltree
        st = time.time()
        bltree = BallTree(X, leaf_size=leaf_size)
        bl_id_et += time.time() - st
        
        st = time.time()
        for j in range(Query.shape[0]): 
            blind = bltree.query_radius(Query[j:j+1], r=R, return_distance=False)  
        bl_qy_et += time.time() - st
        
        # snn
        st = time.time()
        snn = build_snn_model(X)
        sn_id_et += time.time() - st
        
        st = time.time()
        for j in range(Query.shape[0]): 
            sind = snn.radius_single_query(Query[j], R)
        sn_qy_et += time.time() - st
        
        
    print("brute force 1 query time:", bf1et/ len(Rlist))
    print("brute force 2 query time:", bf2et/ len(Rlist))
    print("KDtree index:", kd_id_et/ len(Rlist))
    print("KDtree query time:", kd_qy_et/ len(Rlist))
    print("Balltree index:", bl_id_et/ len(Rlist))
    print("Balltree query time:", bl_qy_et/ len(Rlist))
    print("SNN index:", sn_id_et/ len(Rlist))
    print("SNN query time:", sn_qy_et/ len(Rlist))
    
    index_time_kd.append( kd_id_et / len(Rlist))
    run_time_bf1.append(bf1et / len(Rlist))
    run_time_bf2.append(bf2et / len(Rlist))
    run_time_kd.append( kd_qy_et / len(Rlist))
    index_time_bl.append( bl_id_et / len(Rlist))
    run_time_bl.append( bl_qy_et / len(Rlist))
    index_time_sn.append(sn_id_et / len(Rlist))
    run_time_sn.append(sn_qy_et / len(Rlist))
    print()


 10%|█         | 1/10 [00:34<05:06, 34.10s/it]

average query returns: 0.0
brute force 1 query time: 4.440850675106049
brute force 2 query time: 1.7097725868225098
KDtree index: 0.0031988024711608887
KDtree query time: 0.7272554636001587
Balltree index: 0.002761542797088623
Balltree query time: 0.6679717302322388
SNN index: 0.0019333958625793457
SNN query time: 0.9706563353538513



 20%|██        | 2/10 [02:03<08:52, 66.53s/it]

average query returns: 0.0
brute force 1 query time: 7.292487263679504
brute force 2 query time: 5.027884602546692
KDtree index: 0.029164373874664307
KDtree query time: 4.522170901298523
Balltree index: 0.019347310066223145
Balltree query time: 3.0250934958457947
SNN index: 0.005454123020172119
SNN query time: 2.3813045620918274



 30%|███       | 3/10 [04:54<13:18, 114.12s/it]

...

average query returns: 0.0
brute force 1 query time: 45.255326211452484
brute force 2 query time: 36.53483563661575
KDtree index: 0.18263334035873413
KDtree query time: 58.748392939567566
Balltree index: 0.12711012363433838
Balltree query time: 42.70298606157303
SNN index: 0.04510164260864258
SNN query time: 15.528895199298859



 90%|█████████ | 9/10 [1:01:54<11:07, 667.56s/it]

average query returns: 0.0
brute force 1 query time: 55.04029655456543
brute force 2 query time: 45.49281454086304
KDtree index: 0.2175711989402771
KDtree query time: 70.31866812705994
Balltree index: 0.15293854475021362
Balltree query time: 51.99373185634613
SNN index: 0.055715978145599365
SNN query time: 19.120432257652283



100%|██████████| 10/10 [1:20:47<00:00, 484.71s/it]

average query returns: 0.0
brute force 1 query time: 65.93228161334991
brute force 2 query time: 53.79165202379227
KDtree index: 0.24516594409942627
KDtree query time: 80.05057007074356
Balltree index: 0.17285913228988647
Balltree query time: 59.7801952958107
SNN index: 0.06613820791244507
SNN query time: 23.120337188243866



In [7]:
### save the data

index_time_kd = np.array(index_time_kd)
index_time_bl = np.array(index_time_bl) 
index_time_sn = np.array(index_time_sn) 

run_time_bf1 = np.array(run_time_bf1)
run_time_bf2 = np.array(run_time_bf2)
run_time_kd = np.array(run_time_kd)
run_time_bl = np.array(run_time_bl)
run_time_sn = np.array(run_time_sn)

with open('result/query_r/dim/index_time_kd.npy', 'wb') as f:
    np.save(f, index_time_kd)

with open('result/query_r/dim/index_time_bl.npy', 'wb') as f:
    np.save(f, index_time_bl)

with open('result/query_r/dim/index_time_sn.npy', 'wb') as f:
    np.save(f, index_time_sn)
    
with open('result/query_r/dim/run_time_bf1.npy', 'wb') as f:
    np.save(f, run_time_bf1)

with open('result/query_r/dim/run_time_bf2.npy', 'wb') as f:
    np.save(f, run_time_bf2)
    
with open('result/query_r/dim/run_time_kd.npy', 'wb') as f:
    np.save(f, run_time_kd)

with open('result/query_r/dim/run_time_bl.npy', 'wb') as f:
    np.save(f, run_time_bl)
    
with open('result/query_r/dim/run_time_sn.npy', 'wb') as f:
    np.save(f, run_time_sn)
    